In [1]:
import numpy as np # Funciones numéricas
import pandas as pd # Manejo de bases de datos
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# Geo análisis
import libpysal
import geopandas as gpd
import mapclassify

# Widgets interactivos
import ipywidgets as widgets
from ipywidgets import AppLayout, Layout, HTML, VBox, HBox
from IPython.display import clear_output
#from palettable import colorbrewer

In [2]:
#Carga los shapes de departamento y municipio
#ruta_shapes = "../../MGN_2021_COLOMBIA/ADMINISTRATIVO/"
ruta_shapes = "Shape_files/Colombia/Administrativo/"
shape_dpto = gpd.read_file(ruta_shapes + "MGN_AMN_DPTOS.topojson")
shape_mpio = gpd.read_file(ruta_shapes + "MGN_AMN_MPIOS.topojson")
# crea dos subplots con ejes compartidos

#fig, (ax1) = plt.subplots(1, 1, sharex=True, sharey=True)
#shape_dpto.plot(color='white', edgecolor='black', ax = ax1, linewidth=0.2)
#shape_mpio.plot(color='white', ax = ax1, linewidth=0.2, edgecolor='black', facecolor='white')
#shape_mpio.plot(ax=base);
#plt.show();

In [3]:
# Leer el archivo fuente
path = "Datos/"
file = "datos_mm_cancer.csv"
data = pd.read_csv(path + file, sep=";", encoding= "cp1252",
                   decimal=",", thousands=".")
data['Tipo_cancer'] = data['Tipo_cancer'].str.replace('_invasivo', '', regex=False)

# limpiamos caracteres sombrantes de las columnas de texto
for i in data.select_dtypes(include=['object']).columns:
    data[i] = data[i].str.strip()

#cargamos el archivo con los nomrbes iso y lo agregamos
nombre_iso = pd.read_csv(path + "nombres_iso_dptos.csv", sep=";", encoding= "cp1252",
                         dtype={"Codigo_departamento":str})
display(data.head())
#display(nombre_iso.head())

,Año,Tipo_cancer,Indicador,Categoria,Etiqueta,Numerador,Denominador,Valor_crudo,Valor_ajustado
0,2018,Mama,Mortalidad,Nacional,Colombia,2498,24953862,10.010475,10.010475
1,2018,Mama,Mortalidad,Región,Bogotá,430,4167824,10.317134,9.676866
2,2018,Mama,Mortalidad,Región,Atlántica,507,5328956,9.514059,10.873703
3,2018,Mama,Mortalidad,Región,Central,687,5507205,12.474567,11.774871
4,2018,Mama,Mortalidad,Región,Oriental,389,4980341,7.810710,7.624844


In [4]:
def grafico_coropleta(data, cancer, indicador, nivel, variable):
    """ Esta función realizar el gráfico de coropleta de la variable de interes por el nivel sele
    parametros
    cancer = texto del tipo de cancer para filtrar la base
    indicador = texto del indicador a analizar ('Mortalidad', 'PCNR', 'Prevalencia' )
    nivel = texto nivel geografico del gráfico ('Departamento', 'Municipio')
    Variable = texto del nombre de variable a graficar. La variable debe ser numérica
    """
    #Validaciones
    if (cancer not in list(data['Tipo_cancer'].unique())):
        print("El tipo de cáncer solo toma las valores:" + str(data['Tipo_cancer'].unique()))
        return
    elif (indicador not in list(data['Indicador'].unique())):
        print("El Indicador solo toma las valores:" + str(data['Indicador'].unique()))
        return
    #elif (variable not in ['Valor_crudo', 'Valor_ajustado']):
    #    print("La variable debe estar entre: " + str(['Valor_crudo', 'Valor_ajustado']))
    #    return
    elif (nivel not in ['Departamento', 'Municipio']):
        print("El parámetro Nivel solo puede tomar los valores Departamento o Municipio")
        return

    # Crea el dataframe del gráfico
    df = data.loc[(data["Tipo_cancer"] == cancer) &
                  (data["Indicador"] == indicador) &
                  (data["Categoria"] == nivel)].copy()
    
    if nivel == "Departamento":
        df_gra = df.copy()
        df_gra['geo_code'] = [x[:2] for x in df_gra['Etiqueta']] 
        shape_grafico = shape_dpto.copy()
        shape_grafico['geo_code'] = shape_grafico['DPTO_CCDGO']
        color = 'k'
    else:
        df_gra = df.copy()
        df_gra['geo_code'] = [x[:5] for x in df_gra['Etiqueta']]
        shape_grafico = shape_mpio.copy()
        shape_grafico['geo_code'] = shape_grafico['MPIO_CDPMP']
        color = None

    fig, ax = plt.subplots(figsize=(16, 6))
    #fig.subplots_adjust(bottom=0.025, left=0.025, top = 0.975, right=0.975)
    #rows, cols = 2, 2
    
    #crear el dataset temporal
    if nivel == "Municipio":
            shape_dpto.plot(ax=ax, edgecolor='k')
        
    d = df_gra.query("Año == "+ str(2021)).copy()
    c = shape_grafico.merge(d[['geo_code', variable]],
                                how='left',
                                on='geo_code').copy()
    c.fillna({variable:0}, inplace=True)
    c['cero'] = ['Igual a cero' if x == 0 else "Mayor que cero" for x in c[variable]]
        
        
    # calcula en número de grupos
    #grupos = len(mapclassify.classify(c[variable].values, scheme = esquema.lower(), k=grupos).bins)

    c.plot(ax=ax, column = 'cero', categorical=True,
               #scheme = esquema, k = grupos,
               cmap='Blues', legend=True,
               edgecolor = color,
               legend_kwds= {'ncol':1,
                      'bbox_to_anchor':(0.20, 1),
                      'loc':'upper center',
                      'shadow':True,
                      'frameon':True,
                      #'fmt':"{:.0f}", 'interval':True,
                      'edgecolor':'k'})
        
    ax.set_title("Año " + str(2021), fontsize=10)
    #ax[col].axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    #año = año + 1
    
    titulo = indicador + " de cáncer de " + cancer + " a nivel " + nivel + " con Numerador cero"
    plt.suptitle(titulo, fontsize=18, fontweight="bold")
    
    fig.tight_layout()
    
    plt.savefig("Gr_coropleta_" + cancer + "_" + indicador + "_" + nivel + "_" + variable + ".png", format='png', bbox_inches = 'tight');
    plt.show()

In [5]:

def grafico_coropleta_cat(data, cancer, indicador, nivel, variable):
    """ Esta función realizar el gráfico de coropleta de la variable de interes por el nivel sele
    parametros
    cancer = texto del tipo de cancer para filtrar la base
    indicador = texto del indicador a analizar ('Mortalidad', 'PCNR', 'Prevalencia' )
    nivel = texto nivel geografico del gráfico ('Departamento', 'Municipio')
    Variable = texto del nombre de variable a graficar. La variable debe ser numérica
    """
    #Validaciones
    if (cancer not in list(data['Tipo_cancer'].unique())):
        print("El tipo de cáncer solo toma las valores:" + str(data['Tipo_cancer'].unique()))
        return
    elif (indicador not in list(data['Indicador'].unique())):
        print("El Indicador solo toma las valores:" + str(data['Indicador'].unique()))
        return
    #elif (variable not in ['Valor_crudo', 'Valor_ajustado']):
    #    print("La variable debe estar entre: " + str(['Valor_crudo', 'Valor_ajustado']))
    #    return
    elif (nivel not in ['Departamento', 'Municipio']):
        print("El parámetro Nivel solo puede tomar los valores Departamento o Municipio")
        return

    # Crea el dataframe del gráfico
    df = data.loc[(data["Tipo_cancer"] == cancer) &
                  (data["Indicador"] == indicador) &
                  (data["Categoria"] == nivel)].copy()
    
    if nivel == "Departamento":
        df_gra = df.copy()
        df_gra['geo_code'] = [x[:2] for x in df_gra['Etiqueta']] 
        shape_grafico = shape_dpto.copy()
        shape_grafico['geo_code'] = shape_grafico['DPTO_CCDGO']
        color = 'k'
    else:
        df_gra = df.copy()
        df_gra['geo_code'] = [x[:5] for x in df_gra['Etiqueta']]
        shape_grafico = shape_mpio.copy()
        shape_grafico['geo_code'] = shape_grafico['MPIO_CDPMP']
        color = None

    fig, ax = plt.subplots()
    #fig.subplots_adjust(bottom=0.025, left=0.025, top = 0.975, right=0.975)
            
    #crear el dataset temporal
    if nivel == "Municipio":
            shape_dpto.plot(ax=ax, edgecolor='k')
        
    d = df_gra.query("Año == "+ str(2021)).copy()
    c = shape_grafico.merge(d[['geo_code', variable]],
                                how='left',
                                on='geo_code').copy()
    c.fillna({variable:0}, inplace=True)
    c['cero'] = ['Igual a cero' if x == 0 else "Mayor que cero" for x in c[variable]]

    
    #diccionario de color
    ClusterPalette = {'Mayor que cero': '#E0E0E0',
                      'Igual a cero': '#BD0026'}


    # Loop through each attribute type and plot it using the colors assigned in the dictionary
    for ctype, datos in c.groupby('cero'):
    
        # Define the color for each group using the dictionary
        color_grupo = ClusterPalette[ctype]

        # Plot each group using the color defined above
        datos.plot(ax=ax, 
                   color = color_grupo,
                   label = ctype,
                   edgecolor = color)
    
    # Crea los elementos de la legenda, son los items del diccionario de colores
    legend_elements = [Line2D([0], [0], marker='o', 
                                        color='w', 
                                        markerfacecolor = v, 
                                        label = k, 
                                        markersize=8) 
                       for k, v in ClusterPalette.items()]

    # Inserta la leyenda
    ax.legend(handles=legend_elements,
              ncol=1,
              prop={'size': 8}, 
              bbox_to_anchor=(0.83, 0.99), 
              loc='upper center',
              shadow=True, 
              frameon=True,
              edgecolor='k')

    
    #ax.set_title("Año " + str(2021), fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])
    
    #titulo = indicador + " de cáncer de " + cancer + " a nivel " + nivel + " con " + variable.lower() + " cero"
    #plt.suptitle(titulo, fontsize=18, fontweight="bold")
    
    fig.tight_layout()
    
    #plt.savefig("Gr_coropleta_" + cancer + "_" + indicador + "_" + nivel + "_" + variable + ".png", format='png', bbox_inches = 'tight');
    plt.show()

In [6]:
#widget de Tipo de cancer
wdg_cancer = widgets.Dropdown(
    options = data["Tipo_cancer"].unique(),
    value = data["Tipo_cancer"].unique()[0],
    description='Cáncer:',
    disabled=False,
    layout=Layout(width='auto'))

#widget de Indicador
wdg_indicador = widgets.Dropdown(
    options = data["Indicador"].unique(),
    value = data["Indicador"].unique()[0],
    description='Indicador:',
    disabled=False,
    layout=Layout(width='auto'))

# Años
wdg_years = widgets.Dropdown(
    options = np.sort(data["Año"].unique())[::-1],
    value = np.sort(data["Año"].unique())[::-1][0],
    description='Año:',
    disabled=False,
    layout=Layout(width='auto'))

# Nivel
wdg_nivel = widgets.Dropdown(
    options = ['Departamento', 'Municipio'],
    value = 'Departamento',
    description='Nivel:',
    disabled=False,
    layout=Layout(width='auto'))

# variable
wdg_variable = widgets.Dropdown(
    options = ['Numerador', 'Denominador', 'Valor_crudo', 'Valor_ajustado'],
    value = 'Numerador',
    description='Variable:',
    disabled=False,
    layout=Layout(width='auto'))

button = widgets.Button(description="Generar", button_style='info')
output = widgets.Output(layout=Layout(width='auto'))
def on_button_clicked(b):
    
    titulo = (wdg_indicador.value + " de cáncer de " + str(wdg_cancer.value).lower()    + 
              " a nivel " + str(wdg_nivel.value).lower() + " con " + 
              str(wdg_variable.value).lower() + " cero en el año " + str(wdg_years.value))
    header.value = "<h3>" + titulo + "</h3>"
    
    with output:
        clear_output(wait=False,)
        grafico_coropleta_cat(data      = data,
                              cancer    = wdg_cancer.value,
                              indicador = wdg_indicador.value,
                              nivel     = wdg_nivel.value,
                              variable  = wdg_variable.value)
        
        #display(data[(data["Tipo_cancer"] == wdg_cancer.value)  & 
        #             (data["Indicador"] == wdg_indicador.value) &
        #             (data["Año"] == wdg_years.value) &
        #             (data["Categoria"] == wdg_nivel.value)
        #             ] 
        #       )

        
def cancer_change(change):
    wdg_indicador.options = data[data["Tipo_cancer"] == change["new"]]['Indicador'].unique() 

def indicador_change(change):
    wdg_years.options = data[(data["Tipo_cancer"] == wdg_cancer.value) & 
                       (data["Indicador"] == change["new"])]['Año'].unique() 
    
    wdg_years.value =  data[(data["Tipo_cancer"] == wdg_cancer.value) & 
                       (data["Indicador"] == change["new"])]['Año'].max() 
    
    
wdg_cancer.observe(cancer_change, names=["value"])
wdg_indicador.observe(indicador_change, names=["value"])


button.on_click(on_button_clicked)

In [7]:

titulo = ('Realices selecciones de los filtro y de click al botón "generar"')
header = HTML("<h3>" + titulo + "</h3>", layout=Layout(height='auto'))
header.style.text_align='center'

In [8]:
AppLayout(header=header,
          left_sidebar=VBox([wdg_cancer,
                             wdg_indicador,
                             wdg_years,
                             wdg_nivel,
                             wdg_variable,
                             button]),
          center=output,
          right_sidebar=None,
          footer=None,
          #pane_widths=[1.5, 4,0],
          pane_heights=['40px', 4, 0],
          height='600px',
          grid_gap="30px")

AppLayout(children=(HTML(value='<h3>Realices selecciones de los filtro y de click al botón "generar"</h3>', la…